#### Preprocessing data


##### Filtering by specific postal code for spatial map plotting (Tampines, Control Blocks)

In [1]:
# Getting coordinates of polygons
import geopandas as gpd

geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\SG_geojson\\SG.geojson"
geo_data = gpd.read_file(geojson_path)

In [2]:
blocks_of_interest = [
    '819', '818', '817', '816', '815', '814', '813', '812', '811', '810',
    '809', '808', '807', '806', '805', '804', '803', '802', '801', '930',
    '931', '932', '933', '934', '928', '924', '925', '926', '935', '936'
]

polygons = {}

for block in blocks_of_interest:
    matching_features = geo_data[(geo_data['addr_street'].str.contains("Tampines", na=False)) & 
                                (geo_data['addr_housenumber'].str.contains(f"{block}", na=False))]

    if not matching_features.empty:
        polygon = matching_features.iloc[0]['geometry']
        polygons[f'polygon_{block}'] = polygon
        print(polygons[f'polygon_{block}'])
    else:
        print("No matching features found.")

POLYGON ((103.9362778 1.3485539, 103.9362922 1.3485479, 103.9362826 1.3485249, 103.9364634 1.3484499, 103.9364722 1.348471, 103.9365189 1.3484516, 103.9365096 1.3484292, 103.9367888 1.3483134, 103.9367992 1.3483385, 103.936849 1.3483178, 103.9368386 1.3482927, 103.937096 1.3481857, 103.9371224 1.3481967, 103.9371351 1.3481662, 103.9371034 1.348153, 103.9370814 1.3480963, 103.9371041 1.3480874, 103.9370993 1.3480751, 103.9371356 1.3480608, 103.9371293 1.3480446, 103.9372083 1.3480135, 103.9372663 1.3481603, 103.937246 1.3482103, 103.9372824 1.348225, 103.937271 1.3482531, 103.9372351 1.3482386, 103.9372113 1.3482973, 103.93689 1.3484304, 103.9368992 1.3484526, 103.9368593 1.3484692, 103.9368498 1.3484462, 103.9365601 1.3485663, 103.9365736 1.3485987, 103.9365473 1.3486096, 103.9365336 1.3485764, 103.936314 1.3486675, 103.9362828 1.3485923, 103.9362921 1.3485884, 103.9362778 1.3485539))
POLYGON ((103.936479 1.3480664, 103.9365269 1.3480468, 103.9365171 1.348023, 103.9366065 1.3479864, 10

In [3]:
# Getting postal code
postalcode_geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\ADDRPT.geojson"

# Load GeoJSON data into a GeoDataFrame
postalcode_gdf = gpd.read_file(postalcode_geojson_path)

# print(postalcode_gdf.columns)

def extract_postal_codes(geojson_path, road_name_keyword, blocks_of_interest):
    # Filter for road name containing the specified keyword and block numbers of interest
    filtered_gdf = postalcode_gdf[
        (postalcode_gdf['ROAD_NAME'].str.contains(road_name_keyword, case=False, na=False)) &
        (postalcode_gdf['HOUSE_BLK_NO'].isin(blocks_of_interest))
    ]

    # Extract the postal codes
    postal_codes = filtered_gdf['POSTAL_CODE'].dropna().unique()

    return postal_codes

# Extract postal codes
postal_codes = extract_postal_codes(geojson_path, "Tampines", blocks_of_interest)

# Print the results
print("Postal Codes in Tampines for specified blocks:", postal_codes)

Postal Codes in Tampines for specified blocks: ['520802' '520930' '520804' '520812' '520818' '520819' '520924' '520814'
 '520805' '520815' '520809' '520816' '520801' '520932' '520808' '520931'
 '520925' '520813' '520810' '520807' '520806' '520933' '520935' '520934'
 '520926' '520928' '520803' '520936' '520811' '520817']


In [4]:
import geopandas as gpd
import numpy as np

# Define a dictionary to store coordinates
coordinates_dict = {}

# Function to get coordinates by postal code
def get_coordinates_by_postal_code(postal_code):
    postal_data = postalcode_gdf[postalcode_gdf['POSTAL_CODE'] == postal_code]
    if not postal_data.empty:
        # Extract the coordinates of the first matching entry
        longitude = postal_data.geometry.x.values[0]
        latitude = postal_data.geometry.y.values[0]
        return longitude, latitude
    else:
        return None, None

# Fetch coordinates for each postal code and store them
for postal_code in postal_codes:
    coordinates_dict[postal_code] = get_coordinates_by_postal_code(postal_code)

# Extract coordinates and calculate the central point
longitudes = []
latitudes = []

for postal_code, coords in coordinates_dict.items():
    if coords[0] is not None and coords[1] is not None:
        longitudes.append(coords[0])
        latitudes.append(coords[1])
        print(f'Coordinates for postal code {postal_code}: Longitude {coords[0]}, Latitude {coords[1]}')

if longitudes and latitudes:
    avg_longitude = np.mean(longitudes)
    avg_latitude = np.mean(latitudes)
    print(f'\nCentral coordinates:')
    print(f'Longitude: {avg_longitude}, Latitude: {avg_latitude}')
else:
    print('Coordinates for some or all postal codes not found.')

Coordinates for postal code 520802: Longitude 103.93812757525899, Latitude 1.3463698005975508
Coordinates for postal code 520930: Longitude 103.9396147753537, Latitude 1.3459629199217975
Coordinates for postal code 520804: Longitude 103.93737155793019, Latitude 1.3457495695082662
Coordinates for postal code 520812: Longitude 103.93733287743983, Latitude 1.3469540325138003
Coordinates for postal code 520818: Longitude 103.9368382026983, Latitude 1.3479722949672668
Coordinates for postal code 520819: Longitude 103.93680871187003, Latitude 1.3483964840605756
Coordinates for postal code 520924: Longitude 103.94077696595063, Latitude 1.346791005508926
Coordinates for postal code 520814: Longitude 103.93655386457155, Latitude 1.347283603955788
Coordinates for postal code 520805: Longitude 103.93740847819812, Latitude 1.3461154578053443
Coordinates for postal code 520815: Longitude 103.93740900577279, Latitude 1.3473469261127236
Coordinates for postal code 520809: Longitude 103.93647334330858

In [5]:
# Converting x and y to coordinates for latitude/longitude
import rasterio
import numpy as np
import pandas as pd
from pyproj import Transformer
from shapely.geometry import Point

global filtered_df

def preprocessing(file_path):   
    global filtered_df
    
    # Open your GeoTIFF file
    with rasterio.open(file_path) as src:
        array = src.read()
        transform = src.transform
        src_crs = src.crs  # Source CRS
        # dest_crs = 'EPSG:4326'  # WGS 84

        # Create a transformer object to convert from src_crs to dest_crs
        transformer = Transformer.from_crs(src_crs, 'EPSG:4326', always_xy=True)

        # Get arrays of column and row indices
        cols, rows = np.meshgrid(np.arange(array.shape[2]), np.arange(array.shape[1]))
        
        # Convert meshgrid arrays to coordinate arrays using rasterio's method, which are 2D
        xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')
        
        # Flatten the coordinate arrays to pass to transform function
        lon, lat = transformer.transform(np.array(xs).flatten(), np.array(ys).flatten())

        # Create DataFrame and convert to GeoDataFrame
        df = pd.DataFrame({'Longitude': lon, 'Latitude': lat})
        for i, band in enumerate(src.read(masked=True)):
            df[src.descriptions[i]] = band.flatten()

        # # Convert 'SR_QA_AEROSOL' to integer for bitwise operation
        # df['SR_QA_AEROSOL'] = df['SR_QA_AEROSOL'].astype(int)

        # # Filter out pixels with valid aerosol retrieval and high aerosol level
        # # Assuming 'SR_QA_AEROSOL' is the name of the QA aerosol band in the data
        # valid_aerosol = (df['SR_QA_AEROSOL'] & 2) == 2  # Bit 1 must be set for valid retrieval
        # high_aerosol = (df['SR_QA_AEROSOL'] & 192) == 192  # Bits 6-7 must be set to 11 for high aerosol
        # filter_mask = valid_aerosol & high_aerosol
        # df_filtered = df[-filter_mask]

        df_filtered = df
        
        # Scale and offset specific bands
        df_filtered['ST_B6_Celsius'] = df_filtered['ST_B6'] * 0.00341802 + 149 - 273.15
        df_filtered = df_filtered[df_filtered['ST_B6_Celsius'] >= 20]  # Drop rows below 20 degrees Celsius
        
        bands_to_scale = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
        for band in bands_to_scale:
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * 2.75e-05 - 0.2

        additional_scales = {
            'SR_ATMOS_OPACITY': 0.001,
            'ST_ATRAN': 0.0001, 'ST_CDIST': 0.01, 'ST_DRAD': 0.001, 
            'ST_EMIS': 0.0001, 'ST_EMSD': 0.0001, 'ST_QA': 0.01, 
            'ST_TRAD': 0.001, 'ST_URAD': 0.001
        }

        for band, scale in additional_scales.items():
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * scale

        gdf = gpd.GeoDataFrame(df_filtered, geometry=gpd.points_from_xy(df_filtered.Longitude, df_filtered.Latitude))
        gdf.set_crs('EPSG:4326', inplace=True)

        print("Total number of valid pixels: " + str(len(gdf)))
        print(df[['Latitude', 'Longitude']].head())

        gdf = gdf.to_crs('EPSG:3857')

        transformer_2 = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

        avg_longitude_3857, avg_latitude_3857 = transformer_2.transform(avg_longitude, avg_latitude)

        # Define your point of interest and buffer distance in meters
        poi = Point(avg_longitude_3857, avg_latitude_3857)
        desired_radius = 2000
        buffer = poi.buffer(desired_radius)  # Convert meters to degrees approximately

        # Filter points within the buffer
        filtered_gdf = gdf[gdf.geometry.within(buffer)]

        # Save or process your filtered data
        print(f"\nNumber of points within {desired_radius}m radius: {len(filtered_gdf)}")
        #print(filtered_gdf['ST_B10_Celsius'].head())

        filtered_gdf = filtered_gdf.to_crs('EPSG:4326')

    return filtered_gdf

##### Filtering 30m x 30m pixels based on region of interest

##### Using EPSG:3857 allows you to blow up the pixels in metres because the coordinate representation is in metres

In [6]:
import geopandas as gpd
import hvplot.pandas
import pandas as pd
from shapely.geometry import Polygon, box
import panel as pn
from bokeh.palettes import Inferno256
import numpy as np
import logging

# Suppress warnings
logging.getLogger('bokeh').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None  # default='warn'

global within_polygon_gdf

def plot_spatial_map(filtered_gdf): 
    global within_polygon_gdf
    
    filtered_gdf = filtered_gdf.to_crs('epsg:3857')

    # print(filtered_gdf['geometry'])

    # Create pixels as 30m x 30m boxes around each point
    # Assuming each point is at the center of the pixel
    half_width = 15  # half the width of the pixel in meters since the EPSG:3857 coordinate system is in metres
    filtered_gdf['geometry'] = filtered_gdf['geometry'].apply(lambda x: box(x.x - half_width, x.y - half_width, x.x + half_width, x.y + half_width))

    #print(filtered_gdf['geometry'])

    # Create a GeoDataFrame from all polygons and convert CRS to match
    polygon_gdf = gpd.GeoDataFrame({'geometry': list(polygons.values())}, crs='epsg:4326')
    polygon_gdf_3857 = polygon_gdf.to_crs('epsg:3857')

    # Filter points that intersect any polygon
    def intersects_any_polygon(point):
        return any(point.intersects(poly) for poly in polygon_gdf['geometry'])
    
    filtered_gdf['intersects'] = filtered_gdf['geometry'].apply(intersects_any_polygon)

    # Check intersection with any polygon
    within_polygon_gdf = filtered_gdf[filtered_gdf['intersects']].copy()

    # print(polygon_gdf_3857['geometry'])

    # Filter points that intersect any polygon
    filtered_gdf['intersects'] = filtered_gdf['geometry'].apply(
        lambda geom: any(geom.intersects(poly) for poly in polygon_gdf_3857['geometry']))
    within_polygon_gdf = filtered_gdf[filtered_gdf['intersects']].copy()

    # print(within_polygon_gdf['SR_QA_AEROSOL'].unique())

    print("Number of pixels in region of interest: " + str(len(within_polygon_gdf)))

    # Print or use the filtered GeoDataFrame as needed
    # print("\nNumber of points within the region of interest: " + str(len(within_polygon_gdf)))

    # # Print the centroids of the intersected pixels
    # for index, row in within_polygon_gdf.iterrows():
    #     centroid = row['geometry'].centroid
    #     print(f"Longitude: {centroid.x}, Latitude: {centroid.y}")

    # Define a function to select a subset of the color palette
    def select_colors(palette, n):
        return [palette[int(i)] for i in np.linspace(0, len(palette)-1, n)]

    # Create a custom color scale using a continuous palette
    custom_palette = select_colors(Inferno256, 256)  # More colors for smoother transitions

    # Create the heatmap using the centroid points of intersected pixels
    heatmap = within_polygon_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius', cmap=custom_palette, size=5, tiles='OSM', frame_width=700, frame_height=500, colorbar=True, clim=(20, 40))

    # Plot square polygons with the same color mapping as the points
    squares_plot = within_polygon_gdf.hvplot.polygons('geometry', c='ST_B10_Celsius', cmap=custom_palette, alpha=0.5, colorbar=True, clim=(20, 40))

    # Plot the polygon with visible settings
    polygon_plot = polygon_gdf.hvplot(geo=True, color='red', line_width=3, alpha=0.7)

    # Overlay the polygon onto the heatmap
    overlay_map = polygon_plot * heatmap * squares_plot

    # Set up Panel to display the plot
    # pane = pn.panel(overlay_map)

    # pane.show()
    # pane.save(f'C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\MSE-ES-UHI\\2_landsat\\Heatmaps\\{postal_code_112}_{satellite_image}_LST_Filtered.html', embed=True)

    return overlay_map

#### Plotting LST over time

##### Combining GDFs

In [12]:
import geopandas as gpd
import pandas as pd
import os
import zipfile
from datetime import datetime
import logging
import shutil

# Required data is from 2022 - 2024
year = "2021"

# Suppress warnings
logging.getLogger('bokeh').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None  # default='warn'

# Specify the zip file and temporary directory for extraction
zip_file_path = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat7\\{year}.zip"
temp_dir = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\temp_extract"

# Create a temporary directory if it doesn't exist
os.makedirs(temp_dir, exist_ok=True)

# Extract the .tif files from the zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Initialize an empty list to hold all the GeoDataFrames
gdfs = []

# Walk through the temporary directory and process each .tif file
for filename in os.listdir(f"{temp_dir}\\{year}"):
    if filename.endswith(".tif"):
        print("Currently processing: " + filename)
        file_path = os.path.join(f"{temp_dir}\\{year}", filename)
        
        # Extract the time period from the filename
        # Assuming filename format is "L8_UTC_YYYYMMDD_hhmmss.tif"
        time_str = filename.split('_')[2]
        time_obj = datetime.strptime(time_str, "%Y%m%d")
        
        # Load and preprocess the GeoDataFrame
        gdf = preprocessing(file_path)
        gdf['time'] = time_obj  # Append the datetime object as a new column
        
        # Append the processed GeoDataFrame to the list
        gdfs.append(gdf)

# Combine all GeoDataFrames into one
combined_gdf = pd.concat(gdfs)

shutil.rmtree(f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\temp_extract")

# Use the combined GeoDataFrame as needed
print(combined_gdf)

Currently processing: L7_UTC_20210321_022912.tif
Total number of valid pixels: 672563
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 2000m radius: 600
Currently processing: L7_UTC_20210406_022754.tif
Total number of valid pixels: 688302
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 2000m radius: 707
Currently processing: L7_UTC_20210422_022633.tif
Total number of valid pixels: 550010
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 2000m radius: 11788
Currently processing: L7_UTC_20210508_022513.tif
Total number of valid pixels: 19223
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.59029

In [47]:
# import geopandas as gpd
# import hvplot.pandas
# import holoviews as hv
# from bokeh.palettes import Turbo256  # Import a predefined Bokeh palette

# # Assuming 'gdf' is your preloaded GeoDataFrame
# combined_gdf = combined_gdf.to_crs(epsg=3857)  # Convert to Web Mercator for better mapping support

# # Define a function to select a subset of the color palette
# def select_colors(palette, n):
#     return [palette[int(i)] for i in np.linspace(0, len(palette)-1, n)]

# # Create a custom color scale using a continuous palette
# custom_palette = select_colors(Turbo256, 256)  # More colors for smoother transitions

# # Create the heatmap
# heatmap = combined_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius',
#                             cmap=custom_palette, size=5,  # Smaller size for finer detail
#                             tiles='OSM', frame_width=700, frame_height=500,
#                             colorbar=True, clim=(20, 40))

# #file_path = "C:/LocalOneDrive/Documents/Desktop/MTI/UHI-Project/MSE-ES-UHI/MSE-ES-UHI/2_landsat/Heatmaps"

# # Set up Panel to display the plot
# heatmap_panel = hv.save(heatmap, '270524_hvPlot_Land_Surface_Temperature_Map_gradient.html', backend='bokeh')

# # Display the plot in the notebook
# heatmap_panel

##### Spatial plot over time

In [102]:
import panel as pn

# Create an interactive plot with filtering based on the GeoDataFrame
def create_interactive_plot(combined_gdf):
    # Create a list of unique dates sorted
    unique_dates = combined_gdf['time'].dt.strftime('%Y-%m-%d').sort_values().unique()
    # print(f"Unique Dates: {unique_dates}")

    date_index_map = {i + 1: date for i, date in enumerate(unique_dates)}

    # Setup an integer slider to select time periods
    time_slider = pn.widgets.IntSlider(name='Select Time', start=1, end=len(unique_dates), value=1, step=1)

    @pn.depends(time_slider.param.value_throttled)
    def dynamic_map(value):
        selected_date = date_index_map[value]
        selected_datetime = pd.to_datetime(selected_date).date()
        
        # Filter data for the selected time
        filtered_data = combined_gdf[combined_gdf['time'].dt.date == selected_datetime]
        print(f"Displaying plot for " + str(selected_date))
        
        # Call plot_spatial_map for the selected time period
        return plot_spatial_map(filtered_data)

    layout = pn.Column(
        "<br>\nInteractive Land Surface Temperature Map",
        time_slider,
        dynamic_map
    )

    return layout

layout = create_interactive_plot(combined_gdf)
# layout.servable()
pn.serve(layout, show=False, start=True)

Displaying plot for 2022-02-04
Number of pixels in region of interest: 18
Launching server at http://localhost:61948


Displaying plot for 2022-03-08
Number of pixels in region of interest: 165


#### Exporting data to .csv

In [13]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import box

def filter_and_save_data(year_gdf, polygons, output_file):
    # Convert polygons dictionary to a GeoDataFrame, setting CRS to EPSG:4326 and converting to EPSG:3857
    polygon_gdf = gpd.GeoDataFrame({'block_num': list(polygons.keys()), 'geometry': list(polygons.values())}, crs='epsg:4326')
    polygon_gdf = polygon_gdf.to_crs('epsg:3857')

    # Initialize an empty DataFrame to store all filtered data
    all_filtered_data = gpd.GeoDataFrame()

    for date in year_gdf['time'].dt.strftime('%Y-%m-%d').sort_values().unique():
        # Filter data for the specific date
        date_data = year_gdf[year_gdf['time'].dt.strftime('%Y-%m-%d') == date]

        # Convert CRS to EPSG:3857 and create 30m x 30m boxes around each point
        date_data = date_data.to_crs('epsg:3857')
        date_data['geometry'] = date_data['geometry'].apply(
            lambda x: box(x.x - 15, x.y - 15, x.x + 15, x.y + 15))

        # Initialize an empty list to store block numbers for each point
        block_numbers = []

        # Check each point for intersection with any polygon and store the corresponding block number
        for point in date_data['geometry']:
            found = False
            for _, row in polygon_gdf.iterrows():
                if point.intersects(row['geometry']):
                    block_numbers.append(row['block_num'])
                    found = True
                    break
            if not found:
                block_numbers.append(None)

        # Add the block numbers to the date_data
        date_data['block_num'] = block_numbers

        # Filter points that intersect any polygon
        filtered_data = date_data[date_data['block_num'].notnull()].copy()

        # Append the filtered data of this date to the all_filtered_data DataFrame
        all_filtered_data = pd.concat([all_filtered_data, filtered_data], ignore_index=True)

    # Drop the 'geometry' column as it cannot be saved directly in CSV format
    all_filtered_data.drop(columns=['geometry'], inplace=True)

    # Save the aggregated filtered data to a CSV file
    all_filtered_data.to_csv(output_file, index=False)
    print(f"Data successfully exported to {output_file}")

combined_gdf['time'] = pd.to_datetime(combined_gdf['time'])  # Ensure 'time' is a datetime object
output_path = 'C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\FilteredData\\Tampines\\Landsat7\\Tampines_Control_Filtered_2021.csv'
filter_and_save_data(combined_gdf, polygons, output_path)

Data successfully exported to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\FilteredData\Tampines\Landsat7\Tampines_Control_Filtered_2021.csv


##### Codes to combine .csv files for 2022 - 2024

In [14]:
import pandas as pd

# Define the base file path
base_path = r"C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\FilteredData\Tampines\Landsat7"

# File names
files = [
    r"Tampines_Control_Filtered_2021.csv",
    r"Tampines_Control_Filtered_2022.csv"
    # r"Tampines_Control_Filtered_2023.csv",
    # r"Tampines_Control_Filtered_2024.csv"
]

# Read and concatenate the CSV files
df_list = [pd.read_csv(f"{base_path}\\{file_name}") for file_name in files]
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(f"{base_path}\\Tampines_Control_Filtered_2021_to_2022.csv", index=False)

print("Files were successfully concatenated and saved.")

Files were successfully concatenated and saved.
